# Colab 5: Continued Pretraining on a New Language

This notebook shows how to perform continued pretraining using Unsloth to teach the model a new language.

Dataset: using small small subset of https://www.kaggle.com/datasets/abhishek/hindi-oscar-corpus/data

In [1]:
!pip install -q unsloth transformers datasets accelerate bitsandbytes peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.3/351.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 136.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 20.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

In [2]:
import os
os.environ['WANDB_DISABLED'] = 'true'

from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

In [5]:
from datasets import Dataset

file_name = "hindi_small.txt"

# Read lines from your uploaded text file
with open("hindi_small.txt", "r", encoding="utf-8", errors="ignore") as f:
    lines = [line.strip() for line in f if line.strip()]

print("Loaded lines:", len(lines))
print("Example:", lines[0])

# Create a Hugging Face Dataset object
text_ds = Dataset.from_dict({"text": lines})
print(text_ds)

Loaded lines: 100000
Example: शारदा पारा के मिलन चैiक से आज महापौर देवेन्द्र यादव, पार्षद छोटे लाल चैधरी के साथ वार्ड का भ्रमण करते हुए वार्ड की पेयजल, स्वच्छता सफाई, विकास कार्यों का निरीक्षण किया।
Dataset({
    features: ['text'],
    num_rows: 100000
})


In [6]:
# Load model for continued pretraining
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/smollm2-135m',
    dtype=torch.float16,
    load_in_4bit=True,
)
# Small memory helpers
model.gradient_checkpointing_enable()
model.config.use_cache = False

# Make sure we have a pad token (for batching)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Attach LoRA adapters so training is allowed on a quantized model
model = FastLanguageModel.get_peft_model(
    model,
    r=8,                      # smaller rank = lighter/faster
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)
print("Model loaded with LoRA adapters attached.")

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.11.2 patched 30 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Model loaded with LoRA adapters attached.


In [7]:
# Tokenize the dataset for language modeling
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

tokenized_dataset = text_ds.map(tokenize_function, batched=True, remove_columns=['text'])

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [8]:
# Setup Trainer for continued pretraining
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir='./smollm2_continued_pretrain_hi',
    per_device_train_batch_size=2,
    num_train_epochs=1,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=10,
    max_steps=1000,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 2,442,240 of 136,957,824 (1.78% trained)


Step,Training Loss
10,2.197800
20,2.109500
30,2.148700
40,2.093600
50,2.026000
60,2.133400
70,2.107200
80,2.080900
90,2.090200
100,2.159900


TrainOutput(global_step=1000, training_loss=2.039666751861572, metrics={'train_runtime': 553.1589, 'train_samples_per_second': 3.616, 'train_steps_per_second': 1.808, 'total_flos': 333759578112000.0, 'train_loss': 2.039666751861572, 'epoch': 0.02})

In [9]:
# Save the model
model.save_pretrained('./smollm2_continued_pretrain_hi')
tokenizer.save_pretrained('./smollm2_continued_pretrain_hi')
print('Continued pretraining complete.')

Continued pretraining complete.


In [14]:
# Test the continued-pretrained model
print("\nTesting the continued-pretrained model...\n")

# Example Hindi sentence prompt
prompt_hi = "भारत की राजधानी क्या है?"

# Tokenize and generate output
inputs = tokenizer(prompt_hi, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)

# Decode and print
print("Input:", prompt_hi)
print("Model output:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))



Testing the continued-pretrained model...

Input: भारत की राजधानी क्या है?
Model output:
 भारत की राजधानी क्या है? जो हमें हमें हमें हमें हमें हमें हमें हमें हमें ह
